In [81]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

# Read data

- Read data

In [56]:
data = pd.read_csv('C:/Users/Ananya/Downloads/sem 2/DSP/train.csv')

- Show head of data (first 5 rows)

In [57]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Feature Selection


In [58]:
continuous_features = ['LotArea', 'YearBuilt']
categorical_features = ['Neighborhood', 'BldgType']
target = 'SalePrice'
 

# Training

- Create X and y

In [60]:
# Split dataset into features and target
X = data[continuous_features + categorical_features]
y = data[target]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Feature Processing

In [77]:


# Save the encoder objects
joblib.dump(scaler, 'C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/scaler.joblib')
joblib.dump(onehot, 'C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/Encoder.joblib')

# Load the encoder objects
loaded_onehot = joblib.load('C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/Encoder.joblib')
loaded_scaler = joblib.load('C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/scaler.joblib')


# Define pre-processing for categorical features
onehot = OneHotEncoder(handle_unknown='ignore', sparse=False)
onehot.fit(X_train[categorical_features])

# Define pre-processing for continuous features
scaler = StandardScaler()
scaler.fit(X_train[continuous_features])

# Transform the training data
X_train_categorical = onehot.transform(X_train[categorical_features])
X_train_continuous = scaler.transform(X_train[continuous_features])

# Combine processed features
X_train_processed = np.hstack([X_train_categorical, X_train_continuous])




##  Model Building

## Model Training


In [87]:
# Initialize and train the model
model = LinearRegression()
model.fit(X_train_processed, y_train)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')




Mean Squared Error: 2819332002.017123
R^2 Score: 0.6324364928084286


## Model Evaulation

In [89]:


# Preprocess the test set
X_test_categorical = loaded_onehot.transform(X_test[categorical_features])
X_test_continuous = loaded_scaler.transform(X_test[continuous_features])

# Combine processed features
X_test_processed = np.hstack([X_test_categorical, X_test_continuous])

# Make predictions
y_pred = model.predict(X_test_processed)

# predicting the predicted prices
print("predicted prices for the test data:")
print(y_pred)



predicted prices for the test data:
[135616. 318528. 142976. 136768. 232448. 104064. 220992. 132800.  83072.
 141632. 146752. 150080. 128064. 327872. 193216. 141120. 197312. 145344.
 100608. 210816. 136000. 250112. 210176. 140032. 204800. 130624. 201088.
 159680. 193856. 181504. 101760. 327616. 181696. 143424. 254080. 153088.
 135872. 207616. 320576. 176192. 125632. 193344. 159680. 335168. 135744.
  98240. 150336. 137728. 336192. 149120. 135872. 158784. 109440. 205504.
 143296. 214144. 192960. 195840. 150848. 140736. 118912. 145088. 327104.
 257472. 234560. 131712. 157248. 227648. 173632. 173312. 153920. 140352.
 144448. 115392. 336448. 210816. 317824. 315328. 145728. 176064.  99840.
 101568. 151936. 149568. 149568. 126464. 250432. 204224. 143296. 214336.
 162368. 181696. 161728. 187520. 121408. 200896. 118208. 174016. 211584.
 195392. 161984. 207552. 166848. 120640. 266304. 144192. 155392. 317632.
 200192. 160192. 127488. 183488. 132672. 111808. 211904. 153408. 165504.
 157760. 128704

# Model Inference

In [92]:
# Save transformers
joblib.dump(scaler, 'C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/scaler.joblib')
joblib.dump(onehot, 'C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/Encoder.joblib')
joblib.dump(model, 'C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/model.joblib')

#Load transformers
scaler = joblib.load('C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/scaler.joblib')
onehot = joblib.load('C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/Encoder.joblib')

# Load trained model
loaded_model = joblib.load('C:/Users/Ananya/dsp-ananya-gownivari-ravindrareddy/models/model.joblib')

# Load the new data
new_data = pd.read_csv('C:/Users/Ananya/Downloads/sem 2/DSP/test.csv')

# Preprocess the new data
X_new_categorical = categorical_preprocessor.transform(new_data[categorical_features])
X_new_continuous = continuous_preprocessor.transform(new_data[continuous_features])

# Combine processed features
X_new_processed = np.hstack([X_new_categorical, X_new_continuous])

# Make predictions
new_predictions = model.predict(X_new_processed)

# Display predictions
new_data['PredictedSalePrice'] = new_predictions
print(new_data[['PredictedSalePrice']])



      PredictedSalePrice
0               152384.0
1               152896.0
2               197824.0
3               193792.0
4               262080.0
...                  ...
1454             83072.0
1455            102656.0
1456            155456.0
1457            174272.0
1458            174208.0

[1459 rows x 1 columns]
